# Portfolio Performance vs. Benchmark

### Step 1: Import Libraries

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Visualization libraries
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import Image, display

# System libraries
import os
import sys
import logging
import warnings

warnings.filterwarnings("ignore", category=UserWarning)  # Font warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*inplace.*")  # Pandas warnings
logging.getLogger('matplotlib.font_manager').disabled = True

In [2]:
# UDFs
current_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'py')) 
sys.path.append(current_dir)
from quantstats_fix import *
from utils import load_and_filter_data, export_to_excel
qs.extend_pandas()

               QuantStats Compatibility Tool                

Part 1: Directly patching QuantStats package files
------------------------------------------------------------
Found QuantStats utils file at: /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/__init__.py
Creating backup at: /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/__init__.py.bak
Successfully fixed indentation in QuantStats __init__.py file
✓ QuantStats utils file patched successfully

Part 2: Fixing resampling issues
------------------------------------------------------------
Found 1 potential QuantStats installation(s)
Checking /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/core.py
✓ Found 'plot_timeseries' function in /opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/core.py
✓ No 'sum(axis=0)' calls found - may already be fixed
Examining /opt/hostedtoolcache/Python/3.12.3/x

### Step 2: Define Parameters 

#### Dates

In [3]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')  # Report date
start_date = end_date - timedelta(days=5*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2020-05-17 to 2025-05-16
Time span: 1825 days (5.00 years)


#### Report File

In [4]:
output_file = f'portfolio-{datetime.date(end_date)}.xlsx'

# benchmark = 'ADME'
benchmark = pd.read_excel(output_file, sheet_name="benchmark")['Benchmark'].values[0]
print(benchmark)

ADME


#### Risk-free rate (T-bill, %)

In [5]:
# Load and process data
risk_free_df = pd.read_excel(output_file, sheet_name="daily_quotes", index_col=0)['^IRX']
# risk_free_rate  = 0.0433                              # 3‑month T‑Bill
risk_free_rate = risk_free_df.iloc[-1] / 100 

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")

Risk-Free Rate: 0.0424 -- 13 WEEK TREASURY BILL (^IRX)


### Step 3: Read Portfolio Data (Excel)

In [6]:
portfolio_df = pd.read_excel(output_file, sheet_name="equity")

# Convert percentage strings to float values
portfolio_df['Weight'] = portfolio_df['Weight'].replace('%', '', regex=True).astype(float)
portfolio_tickers = portfolio_df["Ticker"].tolist()

print(portfolio_tickers)
display(portfolio_df)

['GE', 'PGR', 'TJX', 'LIN', 'ADP']


,Ticker,Date,Name,Sector,Industry,Country,Website,Market Cap,Enterprise Value,Float Shares,...,52W High,52W Low,50 Day Avg,200 Day Avg,Short Ratio,Short % of Float,Weight,Expected Return,Standard Deviation,Sharpe Ratio
0,GE,2025-05-19,GE Aerospace,Industrials,Aerospace & Defense,United States,https://www.geaerospace.com,250158039040,255093096448,1062600970,...,234.67,150.20,198.98,185.90,1.56,0.01,0.40,0.415894,0.342721,1.213504
1,PGR,2025-05-19,The Progressive Corporation,Financial Services,Insurance - Property & Casualty,United States,https://www.progressive.com,169310289920,171781554176,584025304,...,292.99,201.34,276.07,256.33,1.70,0.01,0.40,0.287302,0.250641,1.146266
2,TJX,2025-05-19,"The TJX Companies, Inc.",Consumer Cyclical,Apparel Retail,United States,https://www.tjx.com,150714990592,156397174784,1114855115,...,135.08,97.53,123.89,120.49,2.14,0.01,0.10,0.195891,0.255341,0.767175
3,LIN,2025-05-19,Linde plc,Basic Materials,Specialty Chemicals,United Kingdom,https://www.linde.com,217196756992,235778342912,469446732,...,487.49,408.65,452.49,454.67,1.99,0.01,0.05,0.177398,0.223635,0.793246
4,ADP,2025-05-19,"Automatic Data Processing, Inc.",Technology,Software - Application,United States,https://www.adp.com,131005554688,131325648896,405058134,...,322.90,231.27,299.02,292.56,2.76,0.01,0.05,0.180376,0.226907,0.794936


### Step 4: Download Returns

In [7]:
# stock_returns = qs.utils.download_returns(ticker=portfolio_tickers, period="5y").dropna()
stock_quotes = load_and_filter_data('../data/datasets/daily_stock_quotes.csv', portfolio_tickers, start_date, end_date)
stock_returns = np.log(stock_quotes / stock_quotes.shift(1)).dropna()

benchmark_quotes = load_and_filter_data('../data/datasets/daily_benchmark_quotes.csv', benchmark, start_date, end_date)
benchmark_returns = np.log(benchmark_quotes / benchmark_quotes.shift(1)).dropna()

# Display summary statistics for all assets
print("\nSharpe Ratios for individual assets:")
sharpes = {col: qs.stats.sharpe(stock_returns[col]) for col in stock_returns.columns}
for ticker, sharpe in sharpes.items():
    print(f"{ticker}: {sharpe:.4f}")

display(stock_returns.head())

Found 5 of 5 tickers in ../data/datasets/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in ../data/datasets/daily_benchmark_quotes.csv
Missing tickers: []

Sharpe Ratios for individual assets:
ADP: 0.8451
GE: 1.1871
LIN: 0.8422
PGR: 1.1345
TJX: 0.8279


,ADP,GE,LIN,PGR,TJX
Date,,,,,
2020-05-19,-0.024483,-0.009551,-0.010443,-0.009857,0.009846
2020-05-20,0.012522,0.033195,0.029694,-0.013049,0.011857
2020-05-21,-0.010855,0.009240,-0.011905,0.001769,0.065577
2020-05-22,0.011514,-0.010842,-0.014577,-0.003986,-0.016496
2020-05-26,0.029618,0.059134,0.011133,-0.008019,0.034663


### Step 5: Plot Return Comparisons

In [8]:
# Calculate portfolio returns using weights from Excel file
portfolio_weights = portfolio_df.set_index('Ticker')['Weight'].to_dict()
weighted_returns = pd.DataFrame()

print("\nPortfolio Weights:")
for ticker in portfolio_tickers:
    weight = portfolio_weights.get(ticker, 0)
    print(f"{ticker}: {weight:.2%}")
    if ticker in stock_returns.columns:
        weighted_returns[ticker] = stock_returns[ticker] * weight

# Sum across all weighted returns to get the portfolio return
portfolio_return = weighted_returns.sum(axis=1)

# Create equal-weight portfolio for comparison
equal_weight = 1/len([t for t in portfolio_tickers if t in stock_returns.columns])
equal_weighted_returns = pd.DataFrame()

for ticker in portfolio_tickers:
    if ticker in stock_returns.columns:
        equal_weighted_returns[ticker] = stock_returns[ticker] * equal_weight
        
equal_weight_return = equal_weighted_returns.sum(axis=1)

print("\nPortfolio Performance Summary:")
print(f"Sharpe Ratio (Weighted Portfolio): {qs.stats.sharpe(portfolio_return):.4f}")
print(f"Sharpe Ratio (Equal-Weight): {qs.stats.sharpe(equal_weight_return):.4f}")

plt.figure(figsize=(12, 6))
(1 + portfolio_return).cumprod().plot(label='Weighted Portfolio')
(1 + equal_weight_return).cumprod().plot(label='Equal Weight')
(1 + benchmark_returns).cumprod().plot(label=benchmark)
plt.legend()
plt.title('Performance Comparison')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.show()


Portfolio Weights:
GE: 40.00%
PGR: 40.00%
TJX: 10.00%
LIN: 5.00%
ADP: 5.00%

Portfolio Performance Summary:
Sharpe Ratio (Weighted Portfolio): 1.4870
Sharpe Ratio (Equal-Weight): 1.3907


### Step 6: Generate Reports

#### Portfolio (Weighted) vs Benchmark

In [9]:
portfolio_return = portfolio_return.resample('D').sum()  # Example of valid aggregation
portfolio_return.name = "Weighted Portfolio"

# Generate Report
qs.reports.html(
    portfolio_return,
    benchmark_returns,
    rf=risk_free_rate,
    figsize=(8, 5),
    # output=f'portfolio_vs_{benchmark}.html',
    output=f'portfolio_vs_benchmark-{datetime.date(end_date)}.html',
    title=f'Portfolio vs {benchmark} (Benchmark)',
    benchmark_title=f'{benchmark}',
    download_filename=f'portfolio_vs_{benchmark}.html'
)

qs.reports.full(
    portfolio_return, 
    benchmark_returns,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title=f'Portfolio vs {benchmark}',
    benchmark_title=f'{benchmark}') 

/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

Added download button and removed QuantStats attribution from portfolio_vs_benchmark-2025-05-16.html


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

                           ADME        Strategy
-------------------------  ----------  ----------
Start Period               2020-05-20  2020-05-20
End Period                 2025-05-16  2025-05-16
Risk-Free Rate             4.24%       4.24%
Time in Market             68.0%       69.0%

Cumulative Return          47.3%       336.0%
CAGR﹪                     5.5%        22.59%

Sharpe                     0.16        1.0
Prob. Sharpe Ratio         35.13%      93.18%
Smart Sharpe               0.16        0.99
Sortino                    0.23        1.46
Smart Sortino              0.22        1.44
Sortino/√2                 0.16        1.03
Smart Sortino/√2           0.16        1.02
Omega                      1.23        1.23

Max Drawdown               -24.34%     -22.12%
Longest DD Days            898         314
Volatility (ann.)          11.3%       17.69%
R^2                        0.23        0.23
Information Ratio          0.06        0.06
Calmar                     0.23        1.

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2022-01-13,2022-07-14,2022-11-22,314,-22.123936,-20.549891
2,2020-06-09,2020-06-26,2020-10-21,135,-14.137785,-13.823077
3,2025-03-03,2025-04-07,2025-05-06,65,-13.444617,-12.891833
4,2024-11-27,2024-12-18,2025-01-29,64,-10.705333,-9.709678
5,2021-05-17,2021-12-01,2022-01-06,235,-10.435218,-9.480028


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.1

#### Portfolio (Weighted) vs Portolio (Equally-Weighted)

In [10]:
# Ensure equal_weight_return and portfolio_return are properly aggregated if resampled
equal_weight_return = equal_weight_return.resample('D').sum() if equal_weight_return.index.freq is None else equal_weight_return
portfolio_return = portfolio_return.resample('D').sum() if portfolio_return.index.freq is None else portfolio_return

# Set the name for the equal weight portfolio
equal_weight_return.name = "Equal Weight Portfolio"

# Generate Report
qs.reports.html(
    portfolio_return,
    equal_weight_return,
    rf=risk_free_rate,
    figsize=(8, 5),
    output=f'portfolio_vs_equal_weight-{datetime.date(end_date)}.html',
    title='Portfolio (Weighted) vs Portolio (Equally-Weighted)',
    benchmark_title="Equal Weight Portfolio",
    download_filename="portfolio_vs_equal_weight.html" 
)

qs.reports.full(
    portfolio_return, 
    equal_weight_return,
    rf=risk_free_rate, 
    figsize=(8, 5), 
    title='Portfolio vs Equal Weight',
    benchmark_title="Equal Weight Portfolio",
    ) 

/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

Added download button and removed QuantStats attribution from portfolio_vs_equal_weight-2025-05-16.html


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, p

                           Equal Weight Portfolio    Strategy
-------------------------  ------------------------  ----------
Start Period               2020-05-19                2020-05-19
End Period                 2025-05-16                2025-05-16
Risk-Free Rate             4.24%                     4.24%
Time in Market             69.0%                     69.0%

Cumulative Return          229.75%                   332.29%
CAGR﹪                     17.93%                    22.43%

Sharpe                     0.88                      1.0
Prob. Sharpe Ratio         89.83%                    92.94%
Smart Sharpe               0.87                      0.98
Sortino                    1.28                      1.45
Smart Sortino              1.26                      1.43
Sortino/√2                 0.91                      1.02
Smart Sortino/√2           0.89                      1.01
Omega                      1.22                      1.22

Max Drawdown               -19.91%      

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2022-01-13,2022-07-14,2022-11-22,314,-22.123936,-20.549891
2,2020-06-09,2020-06-26,2020-10-21,135,-14.137785,-13.823077
3,2025-03-03,2025-04-07,2025-05-06,65,-13.444617,-12.891833
4,2024-11-27,2024-12-18,2025-01-29,64,-10.705333,-9.709678
5,2021-05-17,2021-12-01,2022-01-06,235,-10.435218,-9.480028


/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:555: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:557: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  .resample("A")
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:565: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").apply(_stats.comp)
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.12/site-packages/quantstats/_plotting/wrappers.py:568: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  returns = returns.resample("A").last()
/opt/hostedtoolcache/Python/3.12.3/x64/lib/python3.1